## ImageJ Ops2
A scientific algorithm framework combined with a bevy of image processing algorithms



## Motivations 

The algorithm framework should prioritize:

1. **Ease of Use**: Image processing algorithms should be easy to access and simple to call.

2. **Portability**: ImageJ Ops2 should extend Java's mantra of "write once, run anywhere". Ops should be usable as-is from any SciJava-compatible software, such as ImageJ or KNIME.

3. **Reproducibility**: Algorithms should be deterministic: calling the same op twice with the same arguments must yield the same result.

4. **Extensibility**: Algorithms can operate on any number of arbitrary data structures. The framework should accomodate (and promote) the addition of new algorithms

<div style="text-align: right">Adapted from <a href=https://github.com/imagej/tutorials/blob/master/notebooks/1-Using-ImageJ/2-ImageJ-Ops.ipynb)><code>imagej/tutorials</code></a></div>

### Op(eration)s
An Op is an implementation of an algorithm that declares a set of preconditions:

* Ops **must** be deterministic

* Ops **must** accept specific kinds of inputs and produce specific kinds of outputs
  * An Op can be defined on an image **of** integer values, or can be defined on a more specific type (like a byte)

`<T extends RealType<T>> Img<T> math.add(Img<T> img1, Img<T> img2)`

This is an Op that adds two images whose values exist within $ R^n $, producing another image whose values exist **within the same numerical space**.

### Flavors of Ops

#### Functions

Given inputs $i_1, i_2, ..., i_n$, produce one output $o$ 

Benefits:
* Convenience - output is given to you

`<T extends RealType<T>> Img<T> math.add(Img<T> img1, Img<T> img2)`

This  Op adds two images whose values exist within $ R^n $, **producing another image** whose values exist **within the same numerical space**.

#### Computers
Given inputs $i_1, i_2, ..., i_n$ and preallocated output $o$, compute some value from $i_1, i_2, ..., i_n$ and store it in $o$ 

Benefits:
* Efficiency: The output can be reused across multiple calls to the Op, preventing the repeated creation of Op outputs
* Extensibility: The user can decide how the output should be created

`<T extends RealType<T>> void math.add(Img<T> img1, Img<T> img2, Img<T> output)`

This Op adds two images whose values exist within $ R^n $, **storing the result in a preallocated output** whose values exist **within the same numerical space**.

#### Inplaces

Given inputs $i_1, i_2, ..., i_n$, compute some value from $i_1, i_2, ..., i_n$ and store it in $i_j$ for some $1\le j\le n$.

Benefits:
* Efficiency: No output is created, either by the user *or* by the Op

`<T extends RealType<T>> void math.add(Img<T> io, Img<T> img2)`

This Op adds two images whose values exist within $ R^n $, **storing the result in the first image** (for C or Java users, this is the Op equivalent of `io += img2`)

## Architecture 

TODO: explain what each of these libraries do.

<div style="position: absolute; width: 25%; left: 5%; top: 0px">
    <b>Scijava Types</b>: Algorithms pertaining to type reasoning
    <ul style="padding-left: 5%">
        <li> Answers questions like:
        <ul style="font-size: 75%;padding-left: 5%">
            <li> Is a <code>Double</code> a <code>Number</code>?
            <li> Is an <code>Img&lt;ByteType&gt;</code> a <code>Number</code>?
            <li> Is an <code>Img&lt;ByteType&gt;</code> an <code>Img&lt;IntegerType&gt;</code>? 
        </ul>
    </ul>
</div>


<div style="position: absolute; width: 25%; left: 35%; top: -50px">
    <b>Scijava Ops</b>: A general-purpose framework for dealing with Ops
    <ul style="padding-left: 5%">
        <li> Improves upon the matching framework of ImageJ Ops
    </ul>
</div>

<div style="position: absolute; width: 25%; left: 65%; top: -100px">
    <b>ImageJ Ops2</b>: A collection of Ops <em>pertaining to image processing</em>
    <ul style="padding-left: 5%">
        <li> Contains ported versions of all Ops from ImageJ Ops
    </ul>
</div>

## Using Ops

First, let's load Ops:

In [7]:
// Load the Scijava Ops library from the remote Maven repository.
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
%%classpath add mvn
org.jetbrains.kotlin kotlin-stdlib 1.3.71
net.imagej imagej 2.1.0
net.imagej imagej-ops2 0-SNAPSHOT

In [12]:
// Initialize ImageJ Ops
ij = new net.imagej.ImageJ()
ops = ij.get(org.scijava.ops.OpService.class)

org.scijava.ops.OpService [priority = 0.0]

Ops can now be called with `ops.op(opName)`:

In [17]:
ops.op("math.div") // We want an Op that adds
""

After specifying the name of the Op, we then provide the input objects or types:

In [14]:
in1 = 2 as Double
in2 = 3 as Double

ops.op("math.add").input(in1, in2) // We want our Op to divide 2 and 3
""

Once we have specified the inputs, we then specify the output object or type:

In [15]:
ops.op("math.add").input(in1, in2).outType(Double.class) // We want our Op to give us a Double
""

Finally we ask Ops to find the Op and run it:

In [28]:
ops.op("math.add").input(in1, in2).outType(Double.class).apply() // Run the op with in1 and in2 as arguments

5.0

We can run the same Op multiple times with different inputs:

In [16]:
adder = ops.op("math.add").input(in1, in2).outType(Double.class).function() // Give us a function back
""

In [30]:
a = 0
b = 1

// find first 10 fibonacci numbers
fibonacci = [0, 1] // starter numbers 
for(int i in 1..8){
    sum = adder.apply(a as Double, b as Double)
    fibonacci.add(sum as Integer)
    a = b
    b = sum
}
fibonacci

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

Let's now run some Ops on an image:

In [21]:
import java.net.URL
import org.scijava.io.location.BytesLocation

openImg = {String imgURL, String name -> 
    url = new URL(imgURL)
    bytes = url.openStream().withCloseable{s -> s.readAllBytes()}

    bytesLocation = new BytesLocation(bytes, name)
    return ij.scifio().datasetIO().open(bytesLocation).getImgPlus()
}

script1597866030448$_run_closure1@757ee973

In [25]:
import net.imglib2.type.numeric.real.FloatType
import net.imglib2.img

// open the image
img = openImg("https://imagej.net/images/blobs.gif", "blobs.gif")

// convert image to allow lossless calculations
convertedInput = ops.op("create.img").input(img, new FloatType()).apply()
convertedInput = ops.op("convert.float32").input(img).outType(Img.class).apply()

// display converted input
ij.notebook().display(convertedInput)

unable to resolve class net.imglib2.img @ line 2, column 1.: unable to resolve class net.imglib2.img @ line 2, column 1.

Let's find the edges of the blobs:

In [33]:
import net.imglib2.type.numeric.real.FloatType

filtered = ops.op("create.img").input(convertedInput).apply() // create image with same size and type as convertedInput
ops.op("filter.sobel").input(convertedInput).output(filtered).compute() // apply sobel filter on convertedInput

// display output
ij.notebook().display(filtered)

Let's intensify the edges with a threshold Op:

In [34]:
import net.imglib2.type.logic.BitType

thresholded = ops.op("create.img").input(filtered, new BitType()).apply() // NOTE: this image MUST contain BitTypes
ops.op("threshold.ij1").input(filtered).output(thresholded).compute() // use the ImageJ1 threshold Op

// display output
ij.notebook().display(thresholded)

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAD+CAYAAADVndu7AAAjRklEQVR4Xu2U25LjOJIF9f8/vWvsLk0rHefEBQApKktu5g+N8Agqa2328Xg8/m9FB7tIBZuKhPMZFWxmdbCblXC+qoNdVQWbr/8awTZxeGjpYOd0sKtIOJ9RwWZWBZsVFWxmjWDbkXD+9V8j2CYOD2Ud7JQRbDvuuvNUwWZGB7sVFWxmdbDrSjj/+q8RbEPbC3+MYEsz2HcknHd0sJvRwW5VwvmsDnZdCedfx3+jV9gWHB5SI9jSDPZdCedVHexmdbBblXA+q4NdRwe7MyWc38EItgWHh9AItjSC7awKNpkRbGdVsNkh4XxGB7sZFWzOMIP9O3WwKzo8hDrYKSPYzqpgExnBdkUFmx0Szmd0sJtRwWaXM/DG1TrYNRwerA52Tge7VRVslBFsV1Ww2SHhfFYFmxkVbHa4Cu9dpYNdw+FB6mAXqWCzQwe7rH+FO6sSzndJOJ9VwWZGBZtVd8LbZ6tg03R4kCrYRDrY7XInvL2qgs0OFWxmVbCZUcFmxYxuf8CdM1WwaTo8DCrYZDrY7XQHvLlDBZsdKtjM6mDX0cFuxQi2lZ0D9mdKOJ9weBgknFd0sNvtCry1SwWbHTrYzehg19HBbtYItk4Fm7NUsJlwePgh4byqgs1ZduH+bgnnu3Swm9HBrmoE21kd7CIVbM5SwWbC4eGHuz6mYHOmGezPlHC+Swe7WRVsqjrYrehgl6lgc5aE8wmHhx/u+JiCzd8k4XynCjaZbs/BrqKD3YoOdpEOds5OqyScTzg8/NB9TL05FWwqKth8goTzXTrYKSOyjrcyHexWdLDLdLDL+ifsMwnnE8b/XymCrZNwHlmFe3eWcL5LBztaIWt50xnBdkUHu0xHtXNw30k4b5sdimCrVLBxzsAbd1TBZlUHu8qOI9rjbWUG+xUd7DIdlSaD31Iq2LTMDjnYORVs6Cq8dzcVbFZ1sKvsOLI93s/6V7jz3ONbVQe7SEelqcDvKRVsWkbHIoZDRgWbrJ+Bd++mgs2KDnbZDht26u2VbK6IvhfBPapgE+moNAdZx+8pFWxaRsccwxGjg11l50mnZ3s3FWxmdLCjhHPVqbcV3Hdmqdzj36d0dJuoZaNUsGk6Hn0NFOKI1MEu6w/YUgWbO6pg09XBjhLOVefeZ9h565XsLv8+paPbzLSUcN40PqgQRwYd7FZ3ol02d1TBpmMEW0o4V51777LrjqJym39j1h9UOt6jhHMl4bxpfFAhjgw62M321MFuVcJ5Vwe7zArcoYTzSjNDdb/aKSq72fyV6t/NTkk4pwo2DceDjBRssv6A3erOzn1lF+5XdbBzVuCOM9shnLtO0dlhG+nI5h0636vY2XOwU5KHGmRLT9hF7QFbqmATqWBTcRXeq9ils8dvRbo9B/dnjWBblUSzDtl3DthU7Ow62BV6HVSWDyrNAX+QUsEmUsEmcxe8W/Es+J3ILtyf1cGuoyKbZ3S/0bG7v4OHO3bCh1IVbJwOds4q3Iv22VTcCW9XrMK9FR3suioqjYJ70T6bqjM3VnmoQ/wI5zPwnlLBxulgp6zAHaWCTdVVeK9jBvtVHexmVFSaJ2ypgk1FwnnkCg91gB9Y/RhvRBLOnQo2ygi2mQo2Xbtwf0XC+Q4d7GZVsFmVcO58bQnbyBle9scD/ACtwr2KCjbUwY5GsK1KOF/Rwe6TdLBbkXC+qoJNJuG8YgXuPNSiiAYz2FdVsKEKNjSCbUcFm6//qWCzKuF8VQWbSAWbEx1/gIishPMZFWyi9oAddbCbkXD+9T8VbFZVsFmVcB6pYHOa/PDBEF1sRKeJVLCZVcHm678SzneoYLMq4dzpYHeab/tw4iy8o3SwW/Gsu79NwvmqCjY7VLBRKtic6ls/HjgL7ygVbFYknH/9T8L5qgo2O3SwW93ZbvRRwvkVduCu0sFuRcL51/9UsFlRwWaXEZXmoHKP311SHcxgf4UOdpkOdisSzr/+p4LNrAo2u12he4ffnvTnQxXufYIOdqsSzr/+lHA+o4PdGc4wu8tvT/jff3QRx26tg90Oz77/m3Sw6+hgd6ZVuj3hd5vmH3dzcey2Otjt8opvXGEE2xUVbKo62F1hRLWrwO829D+AsYLNXVWwiezuEM7vbBfuz6hgU9HBLjKC7Q4jVtqi+qgI/5Fwrsxgf4YKNjslnN/RFXhrRgWbSAc7ZxXurRjBdqYvqI+K8B8J567L4P5OFWx2Szi/mzvgzRkd7JQOdnQG3pgxgu2OPeN4UET/k3Cumg68tUMFm91e+a1Vd8G7s0ZUuye8TWfhnRkj2Fb22BUdD4roHxWVZgbeXZFwvlvC+Z3MYJ/tsZt1B7xJI7KW8xkd7KiDXdHxoIj+UZHNX+G92Z2uhPPdEs7vpIOd0sFuxlV4T+lg51o2XR3sqINd0fGgiP5Rkc0PeEcZwXbG3fecCjZ30cHOGcF2xll4R+lgF+1w3tXBjkawLagPMlRk8wPeiXSwm3H3PaWD3V1UsMl0sFuxA3edCjaUcD6jgg2NYFswP+p4PaQQHyupYNN15y2lg92dVLDJjGB7JxVsKOF8Rgc76mBXMD7oeD3iEB8rq2DTkXC+ooPdnVSwqepgdycJ586ZnUgHO+pgVzA/qsj2xIdaKth0JJzPGMH2birYdHSwu4MKNs6ZnUwFG6pgUzQ/TCo9785IOO9KOO8YwfaOKth0VbC5gwo2SsL5ioRzqmBTdHjg3R9UWjazKth0VLCJrMCdu6pg01XB5g4Szp2E8xUJ5657hW3R4aGlgs2sCjZdFWyUVbh3VxVsuirYvFsFGyfhfFUSzQhvNRweyirYrKhgM2NEtVPwO3dWwaargs27VbBRKtjscBbeaTg8lFWwWVHBZsUd8OanqGDTVcHmnSrYOBVsdjgDbzQdHsoq2KyoYLPqDLyx07PvP1Ww6apg8y4VbJwOdjvswN1Jh4eyCjYrKtjssgN3V81gv0sFm6oOdu9SwcapYLPSKzPYLzg8lFWwqaj2HOzOlHC+6gy8sSLhvKOD3TtUsIlUsHEd4c4NHB7KKtjMqmDzya7AW7MSzjs62F2tg51TwcZ1Du6+2eGhrILNjA52n+wqvDejgk1VB7urVbCJVGTzKvzWLjPQjweqOth1dbD7VCMqzRPe7epgl+lg9w4VbJyKbN6B35t1hpf98WBVB7uODnafagTbbI/NjBFslRFsZ1y9RTh3OirNwcy9riu83BkPd3Swq+pg94k62CkVbGatwJ1sj23VDPaRCjZORTZ/wltRz67qDv7cGo93dLCr6GD3qSrYOBVsVt0Jb1eswj2ngo3Skc0PeKtzt2qVyt6DUdcItpERbD9RBZtIBZtVd8G7FWfgjco9dkpFNn/CW1TBJrICd6LdB6MZI9jSDPafqINdpILNLlfgrYor8FZ0k41Tkc0PeEepYKPMYO8U/RjNeAb8xicawTaTcL7TLtyvGtFpqIKN0tFtnAo2ygi2mdgZg1l3wtufaATbioTzM6zAnY4Odme2nT0H7zgVbGgE24rYG4NZd8G7Z0s4nzWCbUXC+SeqYDPTR7DNdrpNpIINdbCrit0xWHUF3tppF+53jWCbqWDziRLOqYJN1B6wi9qDSsd7TgUbqmDTEftjsMMu3N/lDnizqoNdVQWbT5NwThVssv5Jp8ladk4FG6pgs+DwsE0Hu7PcCW9XVbCpqmDjVLDJVLDpqGBDFWzoLLwT3WKndLCjCjYdsT8G73DXH/f0DPiNigo2VRVsaBXudXYPuFtRwYYq2Ci7cD+7w06pYKNUsFlweLjECtypWoE71X22mQo2FRVssv5s+BsyFWyy/oCdswr3KjfYUQc7JeG8K26MwVnOwBsVHewiHewyFWwqKthk/dnwN2Qq2ETtAbuKEWypgx11sFMSzjuSB4OzXIG3Ih3sMiPYOh3sKirYRO0usm

Scijava Ops and ImageJ Ops2 contain many more Ops; we can use the `help` Op to find out what options exist:

In [35]:
ops.op("help").input("filter.addNoise", ops).outType(String.class).apply() // lookup all noise adding Ops

Available operations:
	net.imagej.ops2.filter.addNoise.NoiseAdders.addNoiseInterval(
	 Inputs:
		net.imglib2.RandomAccessibleInterval<I extends net.imglib2.type.numeric.RealType<I>> input1
		java.lang.Double input2
		java.lang.Double input3
		java.lang.Double input4
		java.lang.Long input5
		net.imglib2.RandomAccessibleInterval<O extends net.imglib2.type.numeric.RealType<O>> mutable1
	 Outputs:
		net.imglib2.RandomAccessibleInterval<O extends net.imglib2.type.numeric.RealType<O>> mutable1
)

	net.imagej.ops2.filter.addNoise.NoiseAdders.addNoiseIntervalSeedless(
	 Inputs:
		net.imglib2.RandomAccessibleInterval<I extends net.imglib2.type.numeric.RealType<I>> input1
		java.lang.Double input2
		java.lang.Double input3
		java.lang.Double input4
		net.imglib2.RandomAccessibleInterval<O extends net.imglib2.type.numeric.RealType<O>> mutable1
	 Outputs:
		net.imglib2.RandomAccessibleInterval<O extends net.imglib2.type.numeric.RealType<O>> mutable1
)


## Extending Ops

Ops can be written as either a class or as a field within a class; each style has benefits and drawbacks
* Ops written as Fields are more concise
* Ops written as Classes can depend on other Ops

### Ops as Classes

Let's first look at how to write an Op as a class:

In [36]:
import java.util.function.BiFunction
import org.scijava.param.Parameter
import org.scijava.struct.ItemIO

@Parameter(key = "in1", itemIO = ItemIO.INPUT)
@Parameter(key = "in2", itemIO = ItemIO.INPUT)
@Parameter(key = "output", itemIO = ItemIO.OUTPUT)
class SampleOp implements BiFunction<Double, Double, Double> {
    @Override
    public Double apply(Double in1, Double in2) {
        return in1 * in2
    }
}

null

We then _"opify"_ the class and register it into the `OpEnvironment`:

In [37]:
info = ops.env().opify(SampleOp.class)
ops.env().register(info, "test.groovyOp")

null

We can now call the Op through Ops:

In [38]:
in1 = 5d
in2 = 3d

result = ops.op("test.groovyOp").input(in1, in2).outType(Double.class).apply()

15.0

Ops written as classes can depend on other ops by using the `@OpDependency` annotation:

In [39]:
import java.util.function.Function
import java.util.List
import org.scijava.ops.OpDependency
import org.scijava.param.Parameter
import org.scijava.struct.ItemIO

@Parameter(key = "inputList", itemIO = ItemIO.INPUT)
@Parameter(key = "output", itemIO = ItemIO.OUTPUT)
class SampleMeanOp implements Function<List<Double>, Double> {
    
    @OpDependency(name = "math.add")
    public Function<List<Double>, Double> sumOp;
    
    @OpDependency(name = "stats.size")
    public Function<List<Double>, Double> sizeOp
    
    @Override
    public Double apply(List<Double> input) {
        return sumOp.apply(input) / sizeOp.apply(input) // mean = sum / size
    }
}

null

Just as before, we _"opify"_ the class and register it into the `OpEnvironment`:

In [40]:
info = ops.env().opify(SampleMeanOp.class)
ops.env().register(info, "test.groovyMean")

null

We can then use our `SampleMeanOp`; Ops will automatically inject the required Op dependencies into our new Op (so long as the `OpService` knows about them)

In [31]:
// samples = [1d, 2d, 4d] as List<Double>

// mean = ops.op("test.groovyMean").input(samples).outType(Double.class).apply()

java.util.function.BiFunction<Double, Double, Double> powOp = {Double base, Double exponent -> Math.pow(base, exponent)}
powOp.apply(2, 3)

8.0

### Ops as Fields

Now let's write an Op as a Field:

In [42]:
import java.util.function.BiFunction
import org.scijava.plugin.Plugin
import org.scijava.ops.core.OpCollection
import org.scijava.ops.OpField

@Plugin(type = OpCollection.class) // tells Ops that this Class contains Field Ops
class OpCollection {

    @OpField(names = "test.groovyPower", params = "base, exponent") // tells Ops that this Field is an Op
    public final BiFunction<Double, Double, Double> powOp = 
        { Double base, Double exponent -> Math.pow(base, exponent)}

}

null

The combination of the `@Plugin` and `@OpField` annotations allows Scijava Ops to discover this Op

## Future Directions 

1. **Improve Accessibility**:
  * Expose Scijava Ops and ImageJ Ops2 in FIJI
  * Support Op autocompletion within popular IDEs

2. **Improve Op Diversity**:
  * Support writing Ops as methods
  * Reduce annotation boilerplate

3. **Op Utilities**:
  * Progress reporting, Logging
  * Op cancellation

## Resources

* Scientific Imaging Forum https://forum.image.sc

* ImageJ Ops Tutorials https://github.com/imagej/tutorials